In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.model_selection import ShuffleSplit, GridSearchCV,train_test_split
from sklearn.model_selection import cross_validate

from sksurv.datasets import load_veterans_lung_cancer
from sksurv.column import encode_categorical
from sksurv.metrics import concordance_index_censored
from sksurv.svm import FastSurvivalSVM,FastKernelSurvivalSVM
from sksurv.kernels import clinical_kernel
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
def score_survival_model(model, X, y):
    prediction = model.predict(X)
    result = concordance_index_censored(y['Status'], y['Survival_in_days'], prediction)
    return result[0]

In [67]:
all_features = list(range(180)) 
clin_demo_comp = list(range(154,180)) 
clin_demo_cyto_gen_comp = list(range(1,180)) 
comp = list(range(163,180)) 
cyto_comp = list(range(85,154))+list(range(163,180)) 
cyto_gen_comp = list(range(1,154))+list(range(163,180)) 
eln_clin_demo_comp = [0]+list(range(154,180)) 
eln_cyto_comp = [0]+list(range(85,154))+list(range(163,180)) 
eln_cyto_gen_comp = list(range(154))+list(range(163,180)) 
eln_gen_comp = list(range(85))+list(range(163,180)) 
gen_comp = list(range(1,85))+list(range(163,180))
clin_comp = list(range(154,161))+list(range(163,180)) 
clin_cyto_comp = list(range(85,161))+list(range(163,180)) 
clin_gen_comp = list(range(1,85))+list(range(154,161))+list(range(163,180)) 
eln_clin_comp = [0]+list(range(154,161))+list(range(163,180))


#Without age
all_features_without_age = list(range(162))+list(range(163,180))
clin_demo_comp_without_age = list(range(154,162))+list(range(163,180))
clin_demo_cyto_gen_comp_without_age = list(range(1,162))+list(range(163,180))
eln_clin_demo_comp_without_age = [0]+list(range(154,162))+list(range(163,180))
            
            
eln_clin_gen = list(range(85))+list(range(154,161))  
eln_demo_gen = list(range(85))+[161,162] 
eln_clin_demo_cyto_gen =list(range(163)) 
eln_clin_demo_cyto = [0]+list(range(85,163))

eln_clin_demo_gen = list(range(85))+list(range(154,163))
eln_clin_demo = [0] + list(range(154,163))
eln_clin = [0] + list(range(154,161))
eln_cyto_gen = list(range(154))
clin_demo_cyto_gen = list(range(1,163))
clin_demo_cyto = list(range(85,163))
clin_demo_gen = list(range(1,85))+list(range(154,163)) 
clin_demo = list(range(154,163)) 
cyto_gen = list(range(1,154))
cyto = list(range(85,154))
gen = list(range(1,85))
clin_gen = list(range(1,85)) + list(range(154,161))  
clin_cyto = list(range(85,161))  
demo_gen = list(range(1,85)) + [161,162]
demo_cyto = list(range(85,154)) + [161,162]

###Without age:

eln_demo_gen_without_age = list(range(85)) + [161]
eln_clin_demo_cyto_gen_without_age = list(range(162))
eln_clin_demo_cyto_without_age = [0] + list(range(85,162))
eln_clin_demo_gen_without_age = list(range(85)) + list(range(154,162))
eln_clin_demo_without_age = [0] + list(range(154,162))
clin_demo_cyto_gen_without_age = list(range(1,162))
clin_demo_cyto_without_age = list(range(85,162))
clin_demo_gen_without_age = list(range(1,85)) + list(range(154,162)) 
clin_demo_without_age = list(range(154,162))
demo_gen_without_age = list(range(1,85)) + [161]
demo_cyto_without_age = list(range(85,154)) + [161]
gen_age = list(range(1,85)) + [162]
eln_comp=[0] + list(range(163,180))
eln_age=[0,162]
eln_gen=[0] + list(range(1,85))
eln_cyto=[0] + list(range(85,154))

dict_features_type_final_comp = dict(zip(("all_features","clin_demo_comp","clin_demo_cyto_gen_comp","comp","cyto_comp","cyto_gen_comp","eln_clin_demo_comp","eln_cyto_comp","eln_cyto_gen_comp",
                                        "eln_gen_comp","gen_comp","clin_comp","clin_cyto_comp","clin_gen_comp","eln_clin_comp","all_features_without_age","clin_demo_comp_without_age",
                                          "clin_demo_cyto_gen_comp_without_age","eln_clin_demo_comp_without_age","eln_clin_gen","eln_demo_gen","eln_clin_demo_cyto_gen","eln_clin_demo_cyto",
                                         "eln_clin_demo_gen","eln_clin_demo","eln_clin","eln_cyto_gen","clin_demo_cyto_gen","clin_demo_cyto","clin_demo_gen","clin_demo","cyto_gen","cyto","gen",
                                          "clin_gen","clin_cyto","demo_gen","demo_cyto","eln_demo_gen_without_age","eln_clin_demo_cyto_gen_without_age","eln_clin_demo_cyto_without_age",
                                          "eln_clin_demo_gen_without_age","eln_clin_demo_without_age","clin_demo_cyto_gen_without_age","clin_demo_cyto_without_age","clin_demo_gen_without_age",
                                          "clin_demo_without_age","demo_gen_without_age","demo_cyto_without_age","gen_age","eln_comp","eln_age","eln_gen","eln_cyto"),
                                         (all_features,clin_demo_comp,clin_demo_cyto_gen_comp,comp,cyto_comp,cyto_gen_comp,eln_clin_demo_comp,eln_cyto_comp,eln_cyto_gen_comp,
                                         eln_gen_comp,gen_comp,clin_comp,clin_cyto_comp,clin_gen_comp,eln_clin_comp,all_features_without_age,clin_demo_comp_without_age,
                                          clin_demo_cyto_gen_comp_without_age,eln_clin_demo_comp_without_age,eln_clin_gen,eln_demo_gen,eln_clin_demo_cyto_gen,eln_clin_demo_cyto,
                                         eln_clin_demo_gen,eln_clin_demo,eln_clin,eln_cyto_gen,clin_demo_cyto_gen,clin_demo_cyto,clin_demo_gen,clin_demo,cyto_gen,cyto,gen,
                                          clin_gen,clin_cyto,demo_gen,demo_cyto,eln_demo_gen_without_age,eln_clin_demo_cyto_gen_without_age,eln_clin_demo_cyto_without_age,
                                          eln_clin_demo_gen_without_age,eln_clin_demo_without_age,clin_demo_cyto_gen_without_age,clin_demo_cyto_without_age,clin_demo_gen_without_age,
                                          clin_demo_without_age,demo_gen_without_age,demo_cyto_without_age,gen_age,eln_comp,eln_age,eln_gen,eln_cyto)))
dicts= dict(zip(("clin_demo_cyto_gen_comp_without_age","eln_clin_demo_comp_without_age","eln_clin_gen","eln_demo_gen","eln_clin_demo_cyto_gen","eln_clin_demo_cyto",
                                         "eln_clin_demo_gen","eln_clin_demo","eln_clin","eln_cyto_gen","clin_demo_cyto_gen","clin_demo_cyto","clin_demo_gen","clin_demo","cyto_gen","cyto","gen",
                                          "clin_gen","clin_cyto","demo_gen","demo_cyto","eln_demo_gen_without_age","eln_clin_demo_cyto_gen_without_age","eln_clin_demo_cyto_without_age",
                                          "eln_clin_demo_gen_without_age","eln_clin_demo_without_age","clin_demo_cyto_gen_without_age","clin_demo_cyto_without_age","clin_demo_gen_without_age",
                                          "clin_demo_without_age","demo_gen_without_age","demo_cyto_without_age","gen_age","eln_comp","eln_age","eln_gen","eln_cyto"
                                          ),
                                         (clin_demo_cyto_gen_comp_without_age,eln_clin_demo_comp_without_age,eln_clin_gen,eln_demo_gen,eln_clin_demo_cyto_gen,eln_clin_demo_cyto,
                                         eln_clin_demo_gen,eln_clin_demo,eln_clin,eln_cyto_gen,clin_demo_cyto_gen,clin_demo_cyto,clin_demo_gen,clin_demo,cyto_gen,cyto,gen,
                                          clin_gen,clin_cyto,demo_gen,demo_cyto,eln_demo_gen_without_age,eln_clin_demo_cyto_gen_without_age,eln_clin_demo_cyto_without_age,
                                          eln_clin_demo_gen_without_age,eln_clin_demo_without_age,clin_demo_cyto_gen_without_age,clin_demo_cyto_without_age,clin_demo_gen_without_age,
                                          clin_demo_without_age,demo_gen_without_age,demo_cyto_without_age,gen_age,eln_comp,eln_age,eln_gen,eln_cyto
                                          )))

In [68]:
df_final = pd.read_table("prognosis_comp_final.tsv")
ci=[]
#df=pd.DataFrame(columns=["feature","ref_CI","permuted_CI","algo","model"])
for key,item in dicts.items():
    for j in range(25):    
        estimator = FastSurvivalSVM(max_iter=1000, tol=1e-6, random_state=j)
        param_grid = {'alpha': 10. ** np.array([-6,-5.5,-5,-4.5,-2.5,-1,0]),'optimizer':["avltree"]}
        cv = ShuffleSplit(n_splits=5,random_state=j)
        gcv = GridSearchCV(estimator, param_grid, scoring=score_survival_model,
                           n_jobs=50, iid=False, refit=True,
                           cv=cv)
        x = df_final.iloc[:,item]
        features = x.columns
        y = np.array(list(zip(df_final.os_status, df_final.os)),dtype=[('Status', '?'), ('Survival_in_days', '<f8')])    
        X_train, X_test, y_train, y_test = train_test_split(pd.DataFrame(x), y, test_size=0.2, random_state=j)
        gcv = gcv.fit(X_train,y_train)
        ref_ci = concordance_index_censored(y_test['Status'], y_test['Survival_in_days'], gcv.predict(X_test))[0]
        print(ref_ci)
        for i in range(4):
            for feature in features:
                X_test_permuted = X_test
                X_test_permuted[feature]=np.random.RandomState(seed=i).permutation(X_test_permuted[feature])      
                permuted_ci = concordance_index_censored(y_test['Status'], y_test['Survival_in_days'], gcv.predict(X_test_permuted))[0]
                df = df.append({'feature': feature, 'ref_CI': ref_ci, 'permuted_CI': permuted_ci, 'algo':'SVM_optimized', 'model': key}, ignore_index=True)

0.655822515631447
0.6606394213844529
0.6891866566716641
0.6662502602540079
0.673002224361741
0.6715026934135299
0.6829015202674881
0.6838308612738377
0.6708084085561826
0.6798827351239997
0.6866432280747025
0.689247311827957
0.6745050858580335
0.6957106838466179
0.6854385826148857
0.6579076504202744
0.6500396498375134
0.6817692863450269
0.6718466099263651
0.6676144578313253
0.6770406291374882
0.6929550811726038
0.6849713253201352
0.6520336913298056
0.6606179580659529
0.6458644110832513
0.664655806957468
0.6910763368315842
0.6757154983263666
0.6696656817502739
0.6732542384084074
0.6774825819840891


/Users/taziy/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:740: ConvergenceWarning: Optimization did not converge: Warning: Desired error not necessarily achieved due to precision loss.
  self.best_estimator_.fit(X, y, **fit_params)


0.700626324317587
0.6655184609943257
0.6717058870137073
0.6913201948198562
0.6851359898798229
0.6685208043624319
0.6865226066915188
0.6870521237048169
0.6670729583841198
0.6561192915895697
0.6874849482218833
0.6616620445634503
0.6716626506024096
0.6717605959578236
0.6856844062282129
0.6744598947285726
0.6499514669505589
0.6460185750342302
0.6528231821499985
0.6578551540947931
0.6929503998001
0.6520283796985858
0.6786295275721258
0.6710730691695033
0.6823744502824766
0.6907952137604387
0.6632117978132833
0.6789794786466999
0.694981130858304
0.6883776091081594
0.6657864720863736
0.6890439140589955
0.6860238867357431
0.6514333655200798
0.662680950974142
0.6820903909448502
0.6662522712058908
0.6707951807228916
0.6717286924340793
0.688456558547193
0.6810904234425328
0.6625544039828413
0.6741285735495471
0.699051004538674
0.6874579959676129
0.722576211894053
0.7108057464084947
0.7105673782410943
0.6992795531907862
0.7305354701628975
0.7193537330175744
0.7003183195189838
0.703177244275414
0.7

/Users/taziy/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:740: ConvergenceWarning: Optimization did not converge: Warning: Desired error not necessarily achieved due to precision loss.
  self.best_estimator_.fit(X, y, **fit_params)


0.6786762868565717
0.6548791620621727
0.651405995816872
0.6688258038930566
0.6708447943603512
0.684282687274087
0.6602438911870089
0.6616591395293558
0.6764022836499693
0.6695762175838077
0.6479117513788847


...

0.6906220215309136
0.7068477470649198


/Users/taziy/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:740: ConvergenceWarning: Optimization did not converge: Warning: Desired error not necessarily achieved due to precision loss.
  self.best_estimator_.fit(X, y, **fit_params)


0.7098789844182551
0.6848891140202301
0.6813863449069395
0.6968772577667175
0.6815849032545982
0.6833574297188755
0.6848330648120085
0.6975283025131968
0.6864875481184697
0.6771456304599681
0.7111961596199211
0.6343114228584125
0.6417656094985118
0.6771067591204398
0.6414580630695559
0.6571743966003785
0.6412472322284279
0.653550310477163
0.6523666334288919
0.6447969367512956
0.6512796133428413
0.6761119891623392
0.6679237824161923
0.6524351182012781
0.669878884437501
0.6649766669303172
0.6235654472638628
0.6367375180756613
0.6608412940515372
0.6196566892990342
0.6592048192771084
0.6401442039273237
0.6593821586268717
0.6572236625029461
0.6385070607759026
0.6496395519556575
0.5987478972926651
0.6276922584568119
0.6641132558720639
0.6257787600698281
0.6071760565718269
0.635050728708814
0.6378946848286199


/Users/taziy/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:740: ConvergenceWarning: Optimization did not converge: Warning: Desired error not necessarily achieved due to precision loss.
  self.best_estimator_.fit(X, y, **fit_params)


0.6310451202791973
0.6380614802626521
0.6263291339830441
0.6363980905073703
0.6416587602783048
0.6151232011999812
0.6391746198703769
0.6342165625247173
0.6206448957624302
0.6166171691570911
0.6459099301597495
0.5979567116126359
0.634441767068273
0.6192154923511302
0.6298927658444274
0.6177233089794957
0.6132777029777374
0.6303139279763771


In [66]:
df.to_csv("SVM_tmp1.cs")